In [194]:
import pandas as pd
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
options = Options()
options.add_argument("--disable-notifications")
options.add_argument("--disable-popup-blocking")
driver = webdriver.Chrome(options=options)


In [195]:
standings_url = 'https://fbref.com/en/comps/9/Premier-League-Stats'

In [196]:
data = requests.get(standings_url)

In [197]:
from bs4 import BeautifulSoup

In [198]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0] #css selector to select table with class name stats_table

links = [l.get('href') for l in standings_table.find_all('a')]
links = [l for l in links if '/squads/' in l]  #only want the links with /squads/ in it
team_urls = [f"https://fbref.com{l}" for l in links] #create list of all the teams urls/links
team_urls

['https://fbref.com/en/squads/18bb7c10/Arsenal-Stats',
 'https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats',
 'https://fbref.com/en/squads/822bd0ba/Liverpool-Stats',
 'https://fbref.com/en/squads/8602292d/Aston-Villa-Stats',
 'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats',
 'https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats',
 'https://fbref.com/en/squads/19538871/Manchester-United-Stats',
 'https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats',
 'https://fbref.com/en/squads/4ba7cbea/Bournemouth-Stats',
 'https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 'https://fbref.com/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats',
 'https://fbref.com/en/squads/fd962109/Fulham-Stats',
 'https://fbref.com/en/squads/47c64c55/Crystal-Palace-Stats',
 'https://fbref.com/en/squads/d3fd31cc/Everton-Stats',
 'https://fbref.com/en/squads/cd051869/Brentford-Stats',
 'https://fbref.com/en/s

In [199]:
all_players=[]
driver.get(team_urls[0]) 
time.sleep(3)
driver.find_element(By.CSS_SELECTOR, 'button[mode=primary]').click() 

for team_url in team_urls:
    driver.get(team_url)    
    time.sleep(2)
    togglep90 =  WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, "stats_defense_9_per_match_toggle")))
    tablename = driver.find_element(By.ID, "all_stats_defense")
    driver.execute_script("arguments[0].scrollIntoView();", tablename)

    time.sleep(2)
    togglep90.click()

    page_source = BeautifulSoup(driver.page_source, 'html.parser')
    tables = page_source.findAll('table', id='stats_defense_9')[0]
    defensivePlayerActions = pd.read_html(str(tables))[0]
    all_players.append(defensivePlayerActions)

driver.quit()


C:\Users\tomas.hehir\AppData\Local\Temp\ipykernel_39044\1924470813.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  defensivePlayerActions = pd.read_html(str(tables))[0]
C:\Users\tomas.hehir\AppData\Local\Temp\ipykernel_39044\1924470813.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  defensivePlayerActions = pd.read_html(str(tables))[0]
C:\Users\tomas.hehir\AppData\Local\Temp\ipykernel_39044\1924470813.py:18: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  defensivePlayerActions = pd.read_html(str(tables))[0]
C:\Users\tomas.hehir\AppData\Local\Temp\ipykernel_39044\1924470813.py:18: FutureWarning: Passing litera

In [201]:
df = pd.concat(all_players)

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0  \
               Player             Nation                Pos   
0      William Saliba             fr FRA                 DF   
1         Declan Rice            eng ENG                 MF   
2         Bukayo Saka            eng ENG                 FW   
3     Martin Ødegaard             no NOR                 MF   
4           Ben White            eng ENG                 DF   
..                ...                ...                ...   
30  Antwoine Hackford            eng ENG                 FW   
31         Sydie Peck            eng ENG                 MF   
32           Ryan One            sct SCO                 FW   
33        Squad Total                NaN                NaN   
34     Opponent Total                NaN                NaN   

   Unnamed: 3_level_0 Unnamed: 4_level_0 Tackles                         \
                  Age                90s     Tkl   TklW Def 3rd Mid 3rd   
0              23-045               36.0    1.00   0.61    0.67    0.31   
1              25-115               33.8    2.34   1.42    1.09    1.04   
2              22-246               31.5    2.03   1.08    0.67    0.73   
3              25-143               32.4    1.42   0.49    0.46    0.65   
4              26-213               31.4    1.46   0.86    0.57    0.70   
..                ...                ...     ...    ...     ...     ...   
30             20-049                0.1    0.00   0.00    0.00    0.00   
31             19-238                0.1    0.00   0.00    0.00    0.00   
32             17-317                0.1    0.00   0.00    0.00    0.00   
33               26.5               36.0   18.80  10.90   10.00    6.58   
34               27.2               36.0   15.80   9.31    7.47    5.89   

            ... Challenges       Blocks             Unnamed: 17_level_0  \
   Att 3rd  ...       Tkl%  Lost Blocks    Sh  Pass                 Int   
0     0.03  ...       75.0  0.17   0.83  0.44  0.39                0.81   
1     0.21  ...       59.3  0.65   1.04  0.30  0.74                1.27   
2     0.63  ...       45.6  0.98   1.33  0.06  1.27                0.44   
3     0.31  ...       29.6  1.17   0.68  0.03  0.65                0.40   
4     0.19  ...       50.0  0.76   1.34  0.64  0.70                0.92   
..     ...  ...        ...   ...    ...   ...   ...                 ...   
30    0.00  ...        NaN  0.00   0.00  0.00  0.00                0.00   
31    0.00  ...        NaN  0.00   0.00  0.00  0.00                0.00   
32    0.00  ...        NaN  0.00   0.00  0.00  0.00                0.00   
33    2.25  ...       46.2  9.14  13.50  4.75  8.78                9.25   
34    2.47  ...       52.1  6.81  10.90  2.72  8.17                7.17   

   Unnamed: 18_level_0 Unnamed: 19_level_0 Unnamed: 20_level_0  \
               Tkl+Int                 Clr                 Err   
0                 1.81                2.08                0.08   
1                 3.61                1.54                0.00   
2                 2.48                0.70                0.03   
3                 1.82                0.15                0.00   
4                 2.39                2.42                0.03   
..                 ...                 ...                 ...   
30                0.00                0.00                0.00   
31                0.00                0.00                0.00   
32                0.00                0.00                0.00   
33               28.10               27.50                0.50   
34               23.00               16.10                0.25   

   Unnamed: 21_level_0  
               Matches  
0              Matches  
1              Matches  
2              Matches  
3              Matches  
4              Matches  
..                 ...  
30             Matches  
31             Matches  
32             Matches  
33                 NaN  
34                 NaN  

[612 rows x 22 columns]

In [202]:
df.columns = ['_'.join(col).strip() for col in df.columns.values]

Index(['Unnamed: 0_level_0_Player', 'Unnamed: 1_level_0_Nation',
       'Unnamed: 2_level_0_Pos', 'Unnamed: 3_level_0_Age',
       'Unnamed: 4_level_0_90s', 'Tackles_Tkl', 'Tackles_TklW',
       'Tackles_Def 3rd', 'Tackles_Mid 3rd', 'Tackles_Att 3rd',
       'Challenges_Tkl', 'Challenges_Att', 'Challenges_Tkl%',
       'Challenges_Lost', 'Blocks_Blocks', 'Blocks_Sh', 'Blocks_Pass',
       'Unnamed: 17_level_0_Int', 'Unnamed: 18_level_0_Tkl+Int',
       'Unnamed: 19_level_0_Clr', 'Unnamed: 20_level_0_Err',
       'Unnamed: 21_level_0_Matches'],
      dtype='object')

In [203]:
df = df.drop(["Challenges_Tkl", "Challenges_Att", "Challenges_Tkl%", "Challenges_Lost", "Unnamed: 21_level_0_Matches"], axis=1)

In [204]:
df

,Unnamed: 0_level_0_Player,Unnamed: 1_level_0_Nation,Unnamed: 2_level_0_Pos,Unnamed: 3_level_0_Age,Unnamed: 4_level_0_90s,Tackles_Tkl,Tackles_TklW,Tackles_Def 3rd,Tackles_Mid 3rd,Tackles_Att 3rd,Blocks_Blocks,Blocks_Sh,Blocks_Pass,Unnamed: 17_level_0_Int,Unnamed: 18_level_0_Tkl+Int,Unnamed: 19_level_0_Clr,Unnamed: 20_level_0_Err
0,William Saliba,fr FRA,DF,23-045,36.0,1.00,0.61,0.67,0.31,0.03,0.83,0.44,0.39,0.81,1.81,2.08,0.08
1,Declan Rice,eng ENG,MF,25-115,33.8,2.34,1.42,1.09,1.04,0.21,1.04,0.30,0.74,1.27,3.61,1.54,0.00
2,Bukayo Saka,eng ENG,FW,22-246,31.5,2.03,1.08,0.67,0.73,0.63,1.33,0.06,1.27,0.44,2.48,0.70,0.03
3,Martin Ødegaard,no NOR,MF,25-143,32.4,1.42,0.49,0.46,0.65,0.31,0.68,0.03,0.65,0.40,1.82,0.15,0.00
4,Ben White,eng ENG,DF,26-213,31.4,1.46,0.86,0.57,0.70,0.19,1.34,0.64,0.70,0.92,2.39,2.42,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,Antwoine Hackford,eng ENG,FW,20-049,0.1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
31,Sydie Peck,eng ENG,MF,19-238,0.1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
32,Ryan One,sct SCO,FW,17-317,0.1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
33,Squad Total,NaN,NaN,26.5,36.0,18.80,10.90,10.00,6.58,2.25,13.50,4.75,8.78,9.25,28.10,27.50,0.50


In [208]:
df = df.set_index("Unnamed: 0_level_0_Player")
df = df.drop(["Squad Total", "Opponent Total"])
df = df.reset_index()
df


,Unnamed: 0_level_0_Player,Unnamed: 1_level_0_Nation,Unnamed: 2_level_0_Pos,Unnamed: 3_level_0_Age,Unnamed: 4_level_0_90s,Tackles_Tkl,Tackles_TklW,Tackles_Def 3rd,Tackles_Mid 3rd,Tackles_Att 3rd,Blocks_Blocks,Blocks_Sh,Blocks_Pass,Unnamed: 17_level_0_Int,Unnamed: 18_level_0_Tkl+Int,Unnamed: 19_level_0_Clr,Unnamed: 20_level_0_Err
0,William Saliba,fr FRA,DF,23-045,36.0,1.00,0.61,0.67,0.31,0.03,0.83,0.44,0.39,0.81,1.81,2.08,0.08
1,Declan Rice,eng ENG,MF,25-115,33.8,2.34,1.42,1.09,1.04,0.21,1.04,0.30,0.74,1.27,3.61,1.54,0.00
2,Bukayo Saka,eng ENG,FW,22-246,31.5,2.03,1.08,0.67,0.73,0.63,1.33,0.06,1.27,0.44,2.48,0.70,0.03
3,Martin Ødegaard,no NOR,MF,25-143,32.4,1.42,0.49,0.46,0.65,0.31,0.68,0.03,0.65,0.40,1.82,0.15,0.00
4,Ben White,eng ENG,DF,26-213,31.4,1.46,0.86,0.57,0.70,0.19,1.34,0.64,0.70,0.92,2.39,2.42,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,Rhys Norrington-Davies,wls WAL,DF,25-016,1.2,0.83,0.83,0.83,0.00,0.00,1.67,0.00,1.67,0.83,1.67,1.67,0.00
568,John Fleck,sct SCO,"MF,DF",32-258,1.0,1.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00
569,Antwoine Hackford,eng ENG,FW,20-049,0.1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
570,Sydie Peck,eng ENG,MF,19-238,0.1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [211]:
df = df[df["Unnamed: 2_level_0_Pos"]== "DF"]
df = df.reset_index()
df = df.drop("index", axis=1)
df

,Unnamed: 0_level_0_Player,Unnamed: 1_level_0_Nation,Unnamed: 2_level_0_Pos,Unnamed: 3_level_0_Age,Unnamed: 4_level_0_90s,Tackles_Tkl,Tackles_TklW,Tackles_Def 3rd,Tackles_Mid 3rd,Tackles_Att 3rd,Blocks_Blocks,Blocks_Sh,Blocks_Pass,Unnamed: 17_level_0_Int,Unnamed: 18_level_0_Tkl+Int,Unnamed: 19_level_0_Clr,Unnamed: 20_level_0_Err
0,William Saliba,fr FRA,DF,23-045,36.0,1.00,0.61,0.67,0.31,0.03,0.83,0.44,0.39,0.81,1.81,2.08,0.08
1,Ben White,eng ENG,DF,26-213,31.4,1.46,0.86,0.57,0.70,0.19,1.34,0.64,0.70,0.92,2.39,2.42,0.03
2,Gabriel Magalhães,br BRA,DF,26-141,32.2,1.43,0.81,0.71,0.68,0.03,1.24,0.93,0.31,0.93,2.36,3.04,0.03
3,Oleksandr Zinchenko,ua UKR,DF,27-145,18.8,2.66,1.60,1.01,1.28,0.37,0.96,0.16,0.80,1.06,3.72,1.38,0.11
4,Jakub Kiwior,pl POL,DF,24-083,10.5,1.90,1.14,1.05,0.76,0.10,1.05,0.29,0.76,0.76,2.67,2.67,0.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,Luke Thomas,eng ENG,DF,22-333,10.2,1.96,0.88,1.27,0.59,0.10,2.25,0.49,1.76,0.98,2.94,2.65,0.00
165,Mason Holgate,eng ENG,DF,27-199,7.6,1.45,0.92,0.66,0.66,0.13,1.45,0.53,0.92,1.32,2.76,5.13,0.00
166,John Egan,ie IRL,DF,31-201,5.4,1.11,0.74,0.74,0.37,0.00,1.85,0.74,1.11,2.41,3.52,7.22,0.19
167,Yasser Larouci,dz ALG,DF,23-128,5.3,3.02,1.51,1.70,0.94,0.38,1.13,0.19,0.94,1.32,4.34,3.77,0.19


In [212]:
df.to_csv("PLdefenderP90.csv")